In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-non

In [1]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointNetClassifier, PointNetLoss, PointNetKAN
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from utils.transformation import Normalization, Rotation, Translation, Reflection, Scale
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [ ]:
# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss

# dataset
transformations = [Normalization(), Rotation(), Translation(normalize=True), Reflection(), Scale()]

train_data = ModelNet(classes, DatasetType.TRAIN, transformations)
validation_data = ModelNet(classes, DatasetType.VALIDATION, transformations)
test_data = ModelNet(classes, DatasetType.TEST, preserve_original=False)

train_loader= DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
# TODO: Más adelante usar alpha para clases imbalanceadas

In [4]:
#<<<<<<< transformation
classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
if DEVICE == "cuda":
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    scheduler = None

base_trainer = PointNetTrainer(
    name="pointnet_base",
    model=classifier,
    optimizer=optimizer,
#=======
base_classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
kan_classifier = PointNetKAN(dim, num_classes, scaling = 3.0).to(DEVICE) 

base_trainer = PointNetTrainer(
    name="pointnet_base",
    model=base_classifier,
    optimizer=optim.Adam(base_classifier.parameters(), lr=learning_rate),
#>>>>>>> main
    scheduler=scheduler,
    criterion=PointNetLoss(alpha=None, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
    device=DEVICE,
    train_loader=train_loader,
    val_loader=val_loader,
    checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
    checkpoint_freq=25
)

# Para este reg_weight = 0, al menos hasta que se decida si utilizar T-net o no.
kan_trainer = PointNetTrainer(
    name="pointnet_kan",
    model=kan_classifier,
    optimizer=optim.Adam(kan_classifier.parameters(), lr=learning_rate),
    scheduler=scheduler,
    criterion=PointNetLoss(alpha=None, gamma=gamma, reg_weight=0, size_average=True).to(DEVICE),
    device=DEVICE,
    train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
    val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
    checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
    checkpoint_freq=25
)

In [6]:
# loss_dict, best_epoch, best_loss, best_acc = base_trainer.fit(epochs=500)
loss_dict, best_epoch, best_loss, best_acc = kan_trainer.fit(epochs=500)
save_loss_dict(loss_dict)
print(f"Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

KeyboardInterrupt: 

# Dataset Testeo

In [ ]:
with torch.no_grad():
    classifier = classifier.eval()
    correct = 0
    
    for pcds, labels in test_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)
        
        # Hacer predicciones
        out, _, _ = classifier(pcds)
    
        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas, acumuladas
        correct += pred_choice.eq(labels.data).cpu().sum().item()

    test_acc = correct / float(len(test_data))
    print("Test Acc:\t", test_acc)

In [ ]:
def test_it(path):
    classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
    classifier.load_state_dict(torch.load(path))

    with torch.no_grad():
        classifier = classifier.eval()
        correct = 0
        
        for pcds, labels in test_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, _ = classifier(pcds)
        
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas, acumuladas
            correct += pred_choice.eq(labels.data).cpu().sum().item()

        test_acc = correct / float(len(test_data))
        print(f"{path}\t\t", test_acc)

# Tests
for i in range(25, 500 + 1, 25):
    path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(i).zfill(4)}.pth")
    test_it(path)
path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
test_it(path)